In [ ]:
!pip install requests
!pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup

## Collect drug info

In [ ]:
def create_soup(letter, num):
  url = f'https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z?alphabet={letter}&page={num}'
  # Gửi yêu cầu đến trang web
  response = requests.get(url)
  # Phân tích nội dung trang web
  soup = BeautifulSoup(response.content, 'html.parser')
  return soup

def get_one_page(soup):
  # Trích xuất các tiêu đề bài viết
  titles_1 = soup.find_all('div', class_='brand_item-info__fXV4x brand_box_shadow_item___lw0f false')
  titles_2 = soup.find_all('div', class_='brand_item-info__fXV4x false')
  drug = {}

  for title in titles_1:
      drug_name = title.p.get_text()
      drug_link = title.a['href']
      drug[drug_name] = drug_link

  for title in titles_2:
      drug_name = title.p.get_text()
      drug_link = title.a['href']
      drug[drug_name] = drug_link

  return drug

In [ ]:
# Sử dụng một vòng lặp để tạo biến chạy từ 'A' đến 'Z'
drug_dict = {}

for letter in range(ord('A'), ord('Z') + 1):
    print(f"Starting collect key {chr(letter)}.")
    soup = create_soup(chr(letter), 1)

    try:
      length = int(soup.find_all('p', class_="css-pqr9s7 text-text-primary")[-1].text)
    except:
      length = 1

    print(f"Need to collect {length} page.")
    drug = get_one_page(soup)

    drug_dict.update(drug)

    if length == 1:
      continue

    for number in range(2, length + 1):
      soup = create_soup(chr(letter), number)

      drug = get_one_page(soup)

      drug_dict.update(drug)

    print(f"Stoping collect key {chr(letter)}.")

Starting collect key W.
Need to collect 1 page.
Starting collect key X.
Need to collect 5 page.
Stoping collect key X.
Starting collect key Y.
Need to collect 2 page.
Stoping collect key Y.
Starting collect key Z.
Need to collect 11 page.
Stoping collect key Z.


In [ ]:
len(drug_dict)

5058

In [ ]:
names = []
links = []
for name, link in drug_dict.items():
  names.append(name)
  links.append(link)

In [ ]:
import pandas as pd

df = pd.DataFrame({"Name": names, "Link": links})
df.head()

,Name,Link
0,Thuốc Actadol 500 Medipharco điều trị các chứn...,/thuoc/actadol-500mg-medipharco-10x10-33851.html
1,Thuốc Allerphast 180mg Mebiphar điều trị viêm ...,/thuoc/allerphast-180mg-4805.html
2,Thuốc A.T Zinc 10mg An Thiên điều trị tiêu chả...,/thuoc/at-zinc-an-thien-10mg-10x10-31306.html
3,Dung dịch uống AtiLude 250mg/5ml An Thiên tiêu...,/thuoc/atilude-250-mg-5-ml-an-thien-6-x5-ong-5...
4,Thuốc Acefalgan 500mg Euvipharm giảm đau nhẹ đ...,/thuoc/acefalgan-500mg-euvipharm-10x10-30343.html


In [ ]:
df.to_csv('drug_info.csv', index=False)

## Collect drug metadata

In [ ]:
url = "https://nhathuoclongchau.com.vn/"